In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
from torch.utils.data import TensorDataset, DataLoader

In [16]:
# Đọc dữ liệu từ file JSON
with open('data.json', 'r') as file:
    data = json.load(file)

In [17]:
# Giả sử dữ liệu là một dictionary với các keys 'o', 'h', 'l', 'c', 'v'
o = np.array(data["o"])
h = np.array(data["h"])
l = np.array(data["l"])
c = np.array(data["c"])
v = np.array(data["v"])

# Chuẩn bị dữ liệu đầu vào và đầu ra
# Đây là một ví dụ đơn giản, bạn cần điều chỉnh cách tạo dữ liệu đầu vào và đầu ra cho phù hợp với yêu cầu của mình
X = np.column_stack((o[:-1], h[:-1], l[:-1], c[:-1], v[:-1]))
y = np.column_stack((o[1:], h[1:], l[1:], c[1:]))

# Chuyển đổi numpy arrays sang torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [18]:
# Tạo DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [19]:
class StockPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StockPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers  # Khai báo num_layers như một thuộc tính của lớp

        # Khởi tạo lớp LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Khởi tạo lớp Linear đầu ra
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Đảm bảo x là 3 chiều (batch, seq_len, features)
        if x.dim() == 2:
            x = x.unsqueeze(0)  # Thêm một chiều batch với kích thước là 1

        # Khởi tạo trạng thái ẩn và trạng thái ô nhớ ban đầu cho LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Lan truyền tiến qua lớp LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Chuyển đầu ra của LSTM qua lớp Linear đầu ra
        out = self.fc(out[:, -1, :])  # Chỉ lấy kết quả của bước thời gian cuối cùng

        # Xử lý đầu ra cho trường hợp không phân batch
        if x.size(0) == 1:
            out = out.squeeze(0)  # Loại bỏ chiều batch để kết quả trả về là 2 chiều

        return out

In [20]:
# Khởi tạo mô hình
input_size = 5  # Số đặc trưng đầu vào
hidden_size = 50  # Kích thước lớp ẩn
num_layers = 2  # Số lớp LSTM
output_size = 4  # Số đặc trưng đầu ra (o, h, l, c cho ngày tiếp theo)

model = StockPriceLSTM(input_size, hidden_size, num_layers, output_size)

# Thiết lập hàm mất mát và bộ tối ưu hóa
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
# Huấn luyện mô hình
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

/Users/khanhle/miniforge3/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/khanhle/miniforge3/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2, 4])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, Loss: 681202.5
Epoch 2/100, Loss: 1296725.625
Epoch 3/100, Loss: 295720.09375
Epoch 4/100, Loss: 830961.0625
Epoch 5/100, Loss: 228694.671875
Epoch 6/100, Loss: 177645.875
Epoch 7/100, Loss: 877381.5
Epoch 8/100, Loss: 1038456.625
Epoch 9/100, Loss: 313304.875
Epoch 10/100, Loss: 274559.21875
Epoch 11/100, Loss: 399414.90625
Epoch 12/100, Loss: 186459.265625
Epoch 13/100, Loss: 504289.9375
Epoch 14/100, Loss: 74090.21875
Epoch 15/100, Loss: 6244.6279296875
Epoch 16/100, Loss: 628761.0
Epoch 17/100, Loss: 709407.8125
Epoch 18/100, Loss: 432234.90625
Epoch 19/100, Loss: 225679.0625
Epoch 20/100, Loss: 344031.53125
Epoch 21/100, Loss: 77683.8046875
Epoch 22/100, Loss: 122528.828125
Epoch 23/100, Loss: 459551.90625
Epoch 24/100, Loss: 139243.34375
Epoch 25/100, Loss: 1052807.75
Epoch 26/100, Loss: 160.1815643310547
Epoch 27/100, Loss: 319136.09375
Epoch 28/100, Loss: 613487.375
Epoch 29/100, Loss: 286480.4375
Epoch 30/100, Loss: 1120.519775390625
Epoch 31/100, Loss: 44237.1445

In [14]:
# Giả sử `model` là mô hình LSTM của bạn đã được huấn luyện
# `last_day_data` là dữ liệu đầu vào cho ngày cuối cùng từ dữ liệu của bạn,
# bao gồm giá mở cửa, cao nhất, thấp nhất, đóng cửa, và khối lượng giao dịch

# Định dạng của last_day_data: [o, h, l, c, v]
last_day_data = np.array([[o[-1], h[-1], l[-1], c[-1], v[-1]]])

# Chuyển đổi dữ liệu sang tensor và thêm chiều batch
last_day_tensor = torch.tensor(last_day_data, dtype=torch.float32).unsqueeze(0)

# Đảm bảo mô hình ở chế độ đánh giá
model.eval()

# Thực hiện dự đoán
with torch.no_grad():
    prediction = model(last_day_tensor)

# Chuyển đổi tensor dự đoán sang numpy array
predicted_values = prediction.numpy()

# In kết quả dự đoán
# predicted_values sẽ chứa dự đoán cho giá mở cửa, cao nhất, thấp nhất, đóng cửa, và khối lượng giao dịch cho ngày hôm sau
print("Dự đoán giá mở cửa, cao nhất, thấp nhất, đóng cửa, khối lượng cho ngày hôm sau:", predicted_values)

Dự đoán giá mở cửa, cao nhất, thấp nhất, đóng cửa cho ngày hôm sau: [69.77768  69.97046  69.507515 69.699524]


In [15]:
torch.save(model, "model.pth")

In [ ]:
model = torch.load("model.pth")